In [11]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity, peak_signal_noise_ratio, mean_squared_error

def calculate_metrics(original_image, encoded_image, message):
    # Calculate BPP (Bits Per Pixel)
    bpp = len(message) * 8 / (original_image.shape[0] * original_image.shape[1])

    # Calculate SSIM (Structural Similarity Index)
    ssim_value, _ = structural_similarity(original_image, encoded_image, full=True, channel_axis=-1)

    # Calculate MSE (Mean Squared Error)
    img1 = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(encoded_image, cv2.COLOR_BGR2GRAY)
    mse = mean_squared_error(img1, img2)
    #mse = np.sum((original_image - encoded_image) ** 2) / float(original_image.size)

    # Calculate PSNR (Peak Signal-to-Noise Ratio)
    psnr = cv2.PSNR(original_image, encoded_image)
    #psnr = peak_signal_noise_ratio(img1, img2)
    #psnr = 10 * np.log10((255 ** 2) / mse)

    return bpp, ssim_value, mse, psnr

def encode_message(img, message, output_path):
    # Convert the message to binary
    binary_message = ''.join(format(ord(char), '08b') for char in message)

    # Check if the message can fit in the image
    if len(binary_message) > img.shape[0] * img.shape[1] * 3:
        raise ValueError("Message is too long for the given image")

    data_index = 0

    # Iterate over each pixel in the image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Iterate over each color channel (BGR)
            for color_channel in range(3):
                # If there is still data to hide
                if data_index < len(binary_message):
                    # Modify the least significant bit of the pixel value
                    img[i, j, color_channel] = img[i, j, color_channel] & ~1 | int(binary_message[data_index])
                    data_index += 1

    # Save the modified image
    cv2.imwrite(output_path, img)

    print("Message encoded successfully!")

def decode_message(encoded_img):
    binary_message = ""

    # Iterate over each pixel in the image
    for i in range(encoded_img.shape[0]):
        for j in range(encoded_img.shape[1]):
            # Iterate over each color channel (BGR)
            for color_channel in range(3):
                # Extract the least significant bit and add it to the binary message
                binary_message += str(encoded_img[i, j, color_channel] & 1)

    # Convert the binary message to ASCII
    decoded_message = "".join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message), 8))

    return decoded_message

filename = './images/Lenna.png'
filename_encoded = './images/Lenna_lsb_encoded.png'
message = "Hello, this is a secret message!"

original_image = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)

# Encode message
encode_message(original_image, message, filename_encoded)

encoded_image = cv2.cvtColor(cv2.imread(filename_encoded), cv2.COLOR_BGR2RGB)

# Decode message
#decoded_message = decode_message(encoded_image)
#print("Decoded message:", decoded_message)

# Calculate metrics
bpp, ssim_value, mse, psnr = calculate_metrics(original_image, encoded_image, message)
print("BPP:", bpp)
print("SSIM:", ssim_value)
print("MSE:", mse)
print("PSNR:", psnr)

Message encoded successfully!
BPP: 0.0004938271604938272
SSIM: 0.7814029003414783
MSE: 239.35560185185184
PSNR: 11.449594013813499


In [ ]:
Message encoded successfully!
BPP: 0.0004938271604938272
SSIM: 0.7814029003414783
MSE: 72.37136188271604
PSNR: 29.535136156208324